# Gesture Recognition

In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.


In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.


In [2]:
np.random.seed(30)
import random as rn

rn.seed(30)
from keras import backend as K
import tensorflow as tf

tf.random.set_seed(30)

2024-02-27 21:36:39.718414: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.


In [3]:
train_doc = np.random.permutation(
    open(
        "/home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/train.csv"
    ).readlines()
)
val_doc = np.random.permutation(
    open(
        "/home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/val.csv"
    ).readlines()
)
batch_size = 20  # experiment with the batch size

## Generator

This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.


In [4]:
def generator(source_path, folder_list, batch_size):
    print("Source path = ", source_path, "; batch size =", batch_size)
    img_idx = [
        10,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19,
        20,
    ]  # create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size  # calculate the number of batches
        for batch in range(num_batches):  # we iterate over the number of batches
            batch_data = np.zeros(
                (batch_size, len(img_idx), 160, 120, 3)
            )  # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros(
                (batch_size, 5)
            )  # batch_labels is the one hot representation of the output
            for folder in range(batch_size):  # iterate over the batch_size
                imgs = os.listdir(
                    source_path + "/" + t[folder + (batch * batch_size)].split(";")[0]
                )  # read all the images in the folder
                for idx, item in enumerate(
                    img_idx
                ):  #  Iterate iver the frames/images of a folder to read them in
                    image = imread(
                        source_path
                        + "/"
                        + t[folder + (batch * batch_size)].strip().split(";")[0]
                        + "/"
                        + imgs[item]
                    ).astype(np.float32)

                    # crop the images and resize them. Note that the images are of 2 different shape
                    # and the conv3D will throw error if the inputs in a batch have different shapes
                    resized_image = resize(image, (160, 120))
                    normalized_image = resized_image / 255.0

                    batch_data[folder, idx, :, :, 0] = normalized_image[:, :, 0]
                    batch_data[folder, idx, :, :, 1] = normalized_image[:, :, 1]
                    batch_data[folder, idx, :, :, 2] = normalized_image[:, :, 2]

                batch_labels[
                    folder, int(t[folder + (batch * batch_size)].strip().split(";")[2])
                ] = 1
            yield batch_data, batch_labels  # you yield the batch_data and the batch_labels, remember what does yield do

        # write the code for the remaining data points which are left after full batches
        if len(folder_list) % batch_size != 0:
            remaining_batch_size = len(folder_list) % batch_size
            batch_data = np.zeros((remaining_batch_size, len(img_idx), 160, 120, 3))
            batch_labels = np.zeros((remaining_batch_size, 5))
            for folder in range(remaining_batch_size):
                imgs = os.listdir(
                    source_path
                    + "/"
                    + t[folder + (num_batches * batch_size)].split(";")[0]
                )
                for idx, item in enumerate(img_idx):
                    image = imread(
                        source_path
                        + "/"
                        + t[folder + (num_batches * batch_size)].strip().split(";")[0]
                        + "/"
                        + imgs[item]
                    ).astype(np.float32)
                    resized_image = resize(image, (160, 120))
                    normalized_image = resized_image / 255.0

                    batch_data[folder, idx, :, :, 0] = normalized_image[:, :, 0]
                    batch_data[folder, idx, :, :, 1] = normalized_image[:, :, 1]
                    batch_data[folder, idx, :, :, 2] = normalized_image[:, :, 2]

                batch_labels[
                    folder,
                    int(t[folder + (num_batches * batch_size)].strip().split(";")[2]),
                ] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.


In [5]:
curr_dt_time = datetime.datetime.now()
train_path = (
    "/home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/train"
)
val_path = "/home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/val"
num_train_sequences = len(train_doc)
print("# training sequences =", num_train_sequences)
num_val_sequences = len(val_doc)
print("# validation sequences =", num_val_sequences)
num_epochs = 10  # choose the number of epochs
print("# epochs =", num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model

Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.


In [6]:
from keras.models import Sequential, Model
from keras.layers import (
    Dense,
    GRU,
    Flatten,
    TimeDistributed,
    Flatten,
    BatchNormalization,
    Activation,
)
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers


# write your model here
def create_model(input_shape, num_classes):
    model = Sequential()
    model.add(
        Conv3D(
            16, (3, 3, 3), padding="same", activation="relu", input_shape=input_shape
        )
    )
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding="same"))
    model.add(Conv3D(32, (3, 3, 3), padding="same", activation="relu"))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding="same"))
    model.add(Conv3D(64, (3, 3, 3), padding="same", activation="relu"))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding="same"))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dense(num_classes, activation="softmax"))
    return model


# Assuming input shape and number of classes
input_shape = (11, 160, 120, 3)
num_classes = 5  # Number of gesture classes

# Create the model
model = create_model(input_shape, num_classes)

2024-02-27 21:36:44.439288: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.


In [7]:
optimiser = "adam"  # write your optimizer
model.compile(
    optimizer=optimiser,
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"],
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 11, 160, 120, 16)  1312      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 6, 80, 60, 16)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 6, 80, 60, 32)     13856     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 3, 40, 30, 32)    0         
 3D)                                                             
                                                                 
 conv3d_2 (Conv3D)           (None, 3, 40, 30, 64)     55360     
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 2, 20, 15, 64)    0

 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 4,986,501
Trainable params: 4,986,501
Non-trainable params: 0
_________________________________________________________________
None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.


In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = (
    "model_init" + "_" + str(curr_dt_time).replace(" ", "").replace(":", "_") + "/"
)

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = (
    model_name
    + "model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5"
)

checkpoint = ModelCheckpoint(
    filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    period=1,
)

LR = ReduceLROnPlateau(
    monitor="val_loss", factor=0.2, patience=5, min_lr=0.001
)  # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.


In [10]:
if (num_train_sequences % batch_size) == 0:
    steps_per_epoch = int(num_train_sequences / batch_size)
else:
    steps_per_epoch = (num_train_sequences // batch_size) + 1

if (num_val_sequences % batch_size) == 0:
    validation_steps = int(num_val_sequences / batch_size)
else:
    validation_steps = (num_val_sequences // batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.


In [11]:
model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
    callbacks=callbacks_list,
    validation_data=val_generator,
    validation_steps=validation_steps,
    class_weight=None,
    workers=1,
    initial_epoch=0,
)

/tmp/ipykernel_8853/2947116171.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(
/tmp/ipykernel_8853/1916305792.py:33: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(


Source path =  /home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/train ; batch size = 20
Epoch 1/10


2024-02-27 21:36:52.165382: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


32/34 [===========================>..] - ETA: 14s - loss: 1.9306 - categorical_accuracy: 0.2219

/tmp/ipykernel_8853/1916305792.py:67: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(


34/34 [==============================] - ETA: 0s - loss: 1.9178 - categorical_accuracy: 0.2232Source path =  /home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/val ; batch size = 20


2024-02-27 21:40:56.665186: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: saving model to model_init_2024-02-2721_36_43.973386/model-00001-1.91782-0.22323-1.51956-0.32000.h5
34/34 [==============================] - 264s 8s/step - loss: 1.9178 - categorical_accuracy: 0.2232 - val_loss: 1.5196 - val_categorical_accuracy: 0.3200 - lr: 0.0010
Epoch 2/10
34/34 [==============================] - ETA: 0s - loss: 1.4442 - categorical_accuracy: 0.3801
Epoch 2: saving model to model_init_2024-02-2721_36_43.973386/model-00002-1.44417-0.38009-1.34697-0.40000.h5
34/34 [==============================] - 237s 7s/step - loss: 1.4442 - categorical_accuracy: 0.3801 - val_loss: 1.3470 - val_categorical_accuracy: 0.4000 - lr: 0.0010
Epoch 3/10
34/34 [==============================] - ETA: 0s - loss: 1.2006 - categorical_accuracy: 0.4827
Epoch 3: saving model to model_init_2024-02-2721_36_43.973386/model-00003-1.20061-0.48265-1.15971-0.53000.h5
34/34 [==============================] - 197s 6s/step - loss: 1.2006 - categorical_accuracy: 0.4827 - val_loss: 1.1597 - val_c